<a href="https://colab.research.google.com/github/anandita-garg/Predicting-Optimal-Locations-For-Solar-Farms/blob/main/year_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

csvs used:

1. filtered_output= solar points from scaredcat
2. india_sf= points from Global-Solar-Power-Tracker dataset that are from india
3. solar_farms.csv= all solar farms found from old dataset
4. unique_best_matches_within_5km= all matches with Global-Solar-Power-Tracker and old dataset
5. updated_unique_best_matches= update with year + if it is in scared cat or not
6. final_unique_best_matches= dropped null year rows.
7. scared cat= dataset with coordinates of different locations, including both solar farm and non solar farm coordinates, with their respective features extracted

In [ ]:
#get all solar locations from scared cat file, essentially extract rows where suitability score =1

import pandas as pd

df = pd.read_csv('ScaredCAT.csv')
filtered_df = df[df['Suitability Score'] == 1]

filtered_df.to_csv('filtered_output.csv', index=False)


In [ ]:
# minimize total distance and maximize unique matches using Hungarian Algorithm

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scipy.optimize import linear_sum_assignment

# Load data
df_india = pd.read_excel("india_sf.xlsx")
df_cat = pd.read_csv('solarfarms.csv')

# Build distance matrix
n_india = len(df_india)
n_cat = len(df_cat)

distance_matrix = np.zeros((n_india, n_cat))

for i, row_india in df_india.iterrows():
    for j, row_cat in df_cat.iterrows():
        dist = geodesic(
            (row_india['Latitude'], row_india['Longitude']),
            (row_cat['latitude'], row_cat['longitude'])
        ).km
        distance_matrix[i, j] = dist

# Solve for minimizing total distance with unique matches
row_ind, col_ind = linear_sum_assignment(distance_matrix)

# Filter matches with distance < 5 km
matches = []
for i, j in zip(row_ind, col_ind):
    dist = distance_matrix[i, j]
    if dist < 5:
        matches.append({
            'India_Latitude': df_india.loc[i, 'Latitude'],
            'India_Longitude': df_india.loc[i, 'Longitude'],
            'ClosestCAT_Latitude': df_cat.loc[j, 'latitude'],
            'ClosestCAT_Longitude': df_cat.loc[j, 'longitude'],
            'Distance_km': round(dist, 3)
        })

# Save to Excel
matched_df = pd.DataFrame(matches)
matched_df.to_excel("unique_best_matches_within_5km.xlsx", index=False)

print(f"{len(matched_df)} unique matches saved to 'unique_best_matches_within_5km.xlsx'")

In [ ]:
# Calculating the number of points that overlap betwwen the scared cat dataset and the best matches dataset.
# This is to see how many solar farm coordinates do we already have the features for

# Load CSVs
india_sf = pd.read_excel("india_sf.xlsx")
filtered_output = pd.read_csv('filtered_output.csv')
unique_best_matches = pd.read_excel('unique_best_matches_within_5km.xlsx')

# Round coordinates for consistent matching
for df in [india_sf, filtered_output]:
    df['Latitude'] = df['Latitude'].round(6)
    df['Longitude'] = df['Longitude'].round(6)

unique_best_matches['ClosestCAT_Latitude'] = unique_best_matches['ClosestCAT_Latitude'].round(6)
unique_best_matches['ClosestCAT_Longitude'] = unique_best_matches['ClosestCAT_Longitude'].round(6)
unique_best_matches['India_Latitude'] = unique_best_matches['India_Latitude'].round(6)
unique_best_matches['India_Longitude'] = unique_best_matches['India_Longitude'].round(6)

filtered_coords = set(zip(filtered_output['Latitude'], filtered_output['Longitude']))

# Mark "1" if the lat-long in unique_best_matches is found in filtered_output
unique_best_matches['in scared cat'] = unique_best_matches.apply(
    lambda row: 1 if (row['ClosestCAT_Latitude'], row['ClosestCAT_Longitude']) in filtered_coords else 0,
    axis=1
)

# Fill in the years for all the solar farm coordinates
merged_year = unique_best_matches.merge(
    india_sf[['Latitude', 'Longitude', 'Start year']],
    left_on=['India_Latitude', 'India_Longitude'],
    right_on=['Latitude', 'Longitude'],
    how='left'
)

unique_best_matches['year'] = merged_year['Start year']

# Final dataset
unique_best_matches.to_csv('updated_unique_best_matches.csv', index=False)

total_overlap = unique_best_matches['in scared cat'].sum()
print(f"Total number of overlapping points with filtered_output: {total_overlap}")


In [ ]:
#check which coordinates have missing years
df = pd.read_csv('updated_unique_best_matches.csv')
missing_years = df['year'].isnull() | (df['year'].astype(str).str.strip() == '')

num_missing = missing_years.sum()
print(f"Number of rows with missing/blank/empty 'year': {num_missing}")


In [ ]:
import pandas as pd

df = pd.read_csv('updated_unique_best_matches.csv')

# Drop rows where 'year' is null/blank/empty
df_cleaned = df[~(df['year'].isnull() | (df['year'].astype(str).str.strip() == ''))]

df_cleaned.to_csv('final_unique_best_matches.csv', index=False)

in_scared_cat_count = df_cleaned['in scared cat'].sum()

print(f"Cleaned CSV saved as 'final_unique_best_matches.csv'")
print(f"Number of rows where 'in scared cat' == 1: {in_scared_cat_count}")
